In [51]:
import pandas as pd
import numpy as np

import altair as alt
import json

### LOAD DATA AND PREPARE

In [2]:
data = pd.read_csv('data/goods-value-2000-2020.csv')

In [3]:
data.head()

,Commodity,Commodity Code,Partner,Partner Code,Partner ISO,Reporter,Reporter Code,Reporter ISO,Trade Flow,Trade Value (US$),Year
0,Animals; live,1,Denmark,208,DNK,Russian Federation,643,RUS,Import,21603580,2017
1,Animals; live,1,Denmark,208,DNK,Russian Federation,643,RUS,Export,100,2017
2,Animals; live,1,Estonia,233,EST,Russian Federation,643,RUS,Import,1373584,2017
3,Animals; live,1,Estonia,233,EST,Russian Federation,643,RUS,Export,1674,2017
4,Animals; live,1,Finland,246,FIN,Russian Federation,643,RUS,Import,1084495,2017


In [4]:
totals = data.groupby(['Partner', 'Year', 'Trade Flow']).agg({
    'Trade Value (US$)': 'sum'
}).reset_index()

In [5]:
totals.head()

,Partner,Year,Trade Flow,Trade Value (US$)
0,Algeria,2000,Export,91969291
1,Algeria,2000,Import,30595
2,Algeria,2001,Export,129432534
3,Algeria,2001,Import,56490
4,Algeria,2002,Export,137641022


In [6]:
data_pct = data.merge(
    totals,
    on = ['Partner', 'Year', 'Trade Flow'],
    how = 'inner',
    suffixes = ["", "_total"]
)

data_pct.loc[:, 'Trade Value Pct'] = (data_pct['Trade Value (US$)'] / data_pct['Trade Value (US$)_total'] * 100).round(2)

In [7]:
data_pct.head()

,Commodity,Commodity Code,Partner,Partner Code,Partner ISO,Reporter,Reporter Code,Reporter ISO,Trade Flow,Trade Value (US$),Year,Trade Value (US$)_total,Trade Value Pct
0,Animals; live,1,Denmark,208,DNK,Russian Federation,643,RUS,Import,21603580,2017,1086635679,1.99
1,Meat and edible meat offal,2,Denmark,208,DNK,Russian Federation,643,RUS,Import,1601932,2017,1086635679,0.15
2,Dairy produce; birds' eggs; natural honey; edi...,4,Denmark,208,DNK,Russian Federation,643,RUS,Import,607834,2017,1086635679,0.06
3,Animal originated products; not elsewhere spec...,5,Denmark,208,DNK,Russian Federation,643,RUS,Import,2246384,2017,1086635679,0.21
4,"Trees and other plants, live; bulbs, roots and...",6,Denmark,208,DNK,Russian Federation,643,RUS,Import,16790015,2017,1086635679,1.55


### TOP-10 GOODS CATEGORIES FOR EACH COUNTRY

In [8]:
def top_goods(group, top = 10):
    subgroup = group.sort_values(by='Trade Value (US$)', ascending=False)
    
    subtop = top if top else len(subgroup)
    
    return subgroup.loc[
        :,
        ['Commodity', 'Commodity Code', 'Trade Value Pct', 'Trade Value (US$)']
    ].head(subtop).reset_index()

In [9]:
top10_goods_data = data_pct.loc[
    :
].groupby([
    'Partner', 'Year', 'Trade Flow'
]).apply(
    top_goods,
    top = 10
)

goods_data = data_pct.loc[
    :
].groupby([
    'Partner', 'Year', 'Trade Flow'
]).apply(
    top_goods,
    top = None
)

In [10]:
top10_goods_data.head()

index  \
Partner Year Trade Flow            
Algeria 2000 Export     0  57512   
                        1  57550   
                        2  57548   
                        3  57549   
                        4  57541   

                                                                   Commodity  \
Partner Year Trade Flow                                                        
Algeria 2000 Export     0  Animal,vegetable fats and oils, cleavage produ...   
                        1               Vehicles other than railway, tramway   
                        2          Nuclear reactors, boilers, machinery, etc   
                        3                   Electrical, electronic equipment   
                        4                                     Iron and steel   

                           Commodity Code  Trade Value Pct  Trade Value (US$)  
Partner Year Trade Flow                                                        
Algeria 2000 Export     0              15            22.09           20314021  
                        1              87            21.48           19758587  
                        2              84            12.84           11805594  
                        3              85             8.00            7355364  
                        4              72             7.25            6670264

In [11]:
top10_goods_data = top10_goods_data.reset_index().rename(columns={'level_3': 'Rank'})
top10_goods_data.loc[:, 'Rank'] += 1
top10_goods_data = top10_goods_data.drop('index', axis=1)

In [12]:
goods_data = goods_data.reset_index().rename(columns={'level_3': 'Rank'})
goods_data.loc[:, 'Rank'] += 1
goods_data = goods_data.drop('index', axis=1)

In [13]:
top10_goods_data.loc[
    top10_goods_data.Partner == 'China'
].sort_values(by=['Year', 'Trade Flow']).head(15)

,Partner,Year,Trade Flow,Rank,Commodity,Commodity Code,Trade Value Pct,Trade Value (US$)
4346,China,2000,Export,1,Iron and steel,72,20.09,883651550
4347,China,2000,Export,2,Aluminium and articles thereof,76,11.86,521513212
4348,China,2000,Export,3,"Ships, boats and other floating structures",89,8.49,373534106
4349,China,2000,Export,4,"Mineral fuels, oils, distillation products, etc",27,7.97,350452163
4350,China,2000,Export,5,"Wood and articles of wood, wood charcoal",44,7.83,344473940
4351,China,2000,Export,6,Plastics and articles thereof,39,5.47,240692881
4352,China,2000,Export,7,Organic chemicals,29,5.46,239940403
4353,China,2000,Export,8,Fertilizers,31,5.35,235255398
4354,China,2000,Export,9,"Pulp of wood, fibrous cellulosic material, was...",47,4.52,198984787
4355,China,2000,Export,10,"Printed books, newspapers, pictures etc",49,3.53,155142710


In [14]:
goods_data.loc[
    goods_data.Partner == 'China'
].sort_values(by=['Year', 'Trade Flow']).head(15)

,Partner,Year,Trade Flow,Rank,Commodity,Commodity Code,Trade Value Pct,Trade Value (US$)
27037,China,2000,Export,1,Iron and steel,72,20.09,883651550
27038,China,2000,Export,2,Aluminium and articles thereof,76,11.86,521513212
27039,China,2000,Export,3,"Ships, boats and other floating structures",89,8.49,373534106
27040,China,2000,Export,4,"Mineral fuels, oils, distillation products, etc",27,7.97,350452163
27041,China,2000,Export,5,"Wood and articles of wood, wood charcoal",44,7.83,344473940
27042,China,2000,Export,6,Plastics and articles thereof,39,5.47,240692881
27043,China,2000,Export,7,Organic chemicals,29,5.46,239940403
27044,China,2000,Export,8,Fertilizers,31,5.35,235255398
27045,China,2000,Export,9,"Pulp of wood, fibrous cellulosic material, was...",47,4.52,198984787
27046,China,2000,Export,10,"Printed books, newspapers, pictures etc",49,3.53,155142710


### TODO: UNIFY COMMODITIES NAMES

### EDA

In [15]:
def subset_top_goods(top_goods_data, country, trade_flow, years = [2000, 2020]):
    subdata = top_goods_data.loc[
        (top_goods_data.Partner == country) & \
        (top_goods_data.Year >= years[0]) & (top_goods_data.Year <= years[1]) &\
        (top_goods_data['Trade Flow'] == trade_flow)
    ]
    
    return subdata

In [16]:
def get_top_goods(top_goods_data, country, trade_flow, years = [2000, 2020], top = 10):
    
    def group_pct(tx):
        years = list(tx['Year'])
        pct_vals = list(tx['Trade Value Pct'])
        
        return [{'year': year, 'pct': pct} for year, pct in zip(years, pct_vals)]
        
    
    subdata = subset_top_goods(top_goods_data, country, trade_flow, years = years)
    
    group = subdata.groupby('Commodity Code')
    
    result = pd.DataFrame({
        'commodity': group.apply(lambda tx: tx['Commodity'].iloc[0]),
        'flow': trade_flow,
        'total': group.apply(lambda tx: tx['Trade Value (US$)'].sum()),
        'records': group.apply(group_pct),
    }).sort_values(by='total', ascending=False).head(top)
    
    return result.reset_index().rename(
        columns={
            'Commodity Code': 'commodity_code'
        }
    ).to_dict(orient='records')

In [17]:
get_top_goods(top10_goods_data, 'Algeria', 'Export', [2010, 2020], top = 3)

[{'commodity_code': 99,
  'commodity': 'Commodities not specified according to kind',
  'flow': 'Export',
  'total': 12399439746,
  'records': [{'year': 2016, 'pct': 64.81},
   {'year': 2017, 'pct': 57.01},
   {'year': 2018, 'pct': 66.78},
   {'year': 2019, 'pct': 53.28},
   {'year': 2020, 'pct': 68.83}]},
 {'commodity_code': 93,
  'commodity': 'Arms and ammunition; parts and accessories thereof',
  'flow': 'Export',
  'total': 3140357961,
  'records': [{'year': 2010, 'pct': 28.97},
   {'year': 2011, 'pct': 46.33},
   {'year': 2012, 'pct': 24.99},
   {'year': 2013, 'pct': 31.84},
   {'year': 2014, 'pct': 1.19},
   {'year': 2015, 'pct': 24.16},
   {'year': 2016, 'pct': 8.43},
   {'year': 2017, 'pct': 0.55},
   {'year': 2018, 'pct': 1.37},
   {'year': 2019, 'pct': 1.85}]},
 {'commodity_code': 87,
  'commodity': 'Vehicles other than railway or tramway rolling-stock, and parts and accessories thereof',
  'flow': 'Export',
  'total': 2861082577,
  'records': [{'year': 2010, 'pct': 9.17},
  

In [18]:
get_top_goods(goods_data, 'Algeria', 'Export', [2010, 2020], top = 3)

[{'commodity_code': 99,
  'commodity': 'Commodities not specified according to kind',
  'flow': 'Export',
  'total': 12399439746,
  'records': [{'year': 2016, 'pct': 64.81},
   {'year': 2017, 'pct': 57.01},
   {'year': 2018, 'pct': 66.78},
   {'year': 2019, 'pct': 53.28},
   {'year': 2020, 'pct': 68.83}]},
 {'commodity_code': 93,
  'commodity': 'Arms and ammunition; parts and accessories thereof',
  'flow': 'Export',
  'total': 3140357961,
  'records': [{'year': 2010, 'pct': 28.97},
   {'year': 2011, 'pct': 46.33},
   {'year': 2012, 'pct': 24.99},
   {'year': 2013, 'pct': 31.84},
   {'year': 2014, 'pct': 1.19},
   {'year': 2015, 'pct': 24.16},
   {'year': 2016, 'pct': 8.43},
   {'year': 2017, 'pct': 0.55},
   {'year': 2018, 'pct': 1.37},
   {'year': 2019, 'pct': 1.85}]},
 {'commodity_code': 87,
  'commodity': 'Vehicles other than railway or tramway rolling-stock, and parts and accessories thereof',
  'flow': 'Export',
  'total': 2861243832,
  'records': [{'year': 2010, 'pct': 9.17},
  

#### AREA CHART FACET

In [19]:
def country_goods_facet(top_goods_data, country, trade_flow, years, top):

    country_data = get_top_goods(top_goods_data, country, trade_flow, years, top = top)

    chart_df = pd.DataFrame([])
    for commodity in country_data:
        df = pd.DataFrame(commodity['records'])
        df.loc[:, 'commodity'] = commodity['commodity']
        df.loc[:, 'commodity_code'] = commodity['commodity_code']

        chart_df = pd.concat([
            chart_df,
            df
        ])

    chart = alt.Chart(chart_df).mark_area().encode(
        alt.X('year:O'),
        alt.Y('pct', scale=alt.Scale(domain=[0, 100])),
        color = alt.Color('commodity:N')
    ).properties(
        width=250,
        height=55
    ).facet(
        facet='commodity_code',
        columns = 1
    )
    
    return chart

In [20]:
alt.hconcat(
    country_goods_facet(top10_goods_data, "China", "Import", years=[2000, 2020], top=5),
    country_goods_facet(top10_goods_data, "China", "Export", years=[2000, 2020], top=5)
)

alt.HConcatChart(...)

In [21]:
alt.hconcat(
    country_goods_facet(goods_data, "China", "Import", years=[2000, 2020], top=5),
    country_goods_facet(goods_data, "China", "Export", years=[2000, 2020], top=5)
)

alt.HConcatChart(...)

In [22]:
country = "Germany"
alt.hconcat(
    country_goods_facet(goods_data, country, "Import", years=[2000, 2020], top=5),
    country_goods_facet(goods_data, country, "Export", years=[2000, 2020], top=5)
)

alt.HConcatChart(...)

In [23]:
country = "Belarus"
alt.hconcat(
    country_goods_facet(goods_data, country, "Import", years=[2000, 2020], top=5),
    country_goods_facet(goods_data, country, "Export", years=[2000, 2020], top=5)
)

alt.HConcatChart(...)

In [24]:
country = "Netherlands"
alt.hconcat(
    country_goods_facet(goods_data, country, "Import", years=[2000, 2020], top=5),
    country_goods_facet(goods_data, country, "Export", years=[2000, 2020], top=5)
)

alt.HConcatChart(...)

### SAVE DATA

In [25]:
len(goods_data)

176767

In [26]:
goods_data.head()

,Partner,Year,Trade Flow,Rank,Commodity,Commodity Code,Trade Value Pct,Trade Value (US$)
0,Algeria,2000,Export,1,"Animal,vegetable fats and oils, cleavage produ...",15,22.09,20314021
1,Algeria,2000,Export,2,"Vehicles other than railway, tramway",87,21.48,19758587
2,Algeria,2000,Export,3,"Nuclear reactors, boilers, machinery, etc",84,12.84,11805594
3,Algeria,2000,Export,4,"Electrical, electronic equipment",85,8.00,7355364
4,Algeria,2000,Export,5,Iron and steel,72,7.25,6670264


In [47]:
def goods_recs(country, flow):
    
    recs = goods_data.loc[
        (goods_data.Partner == country) & (goods_data['Trade Flow'] == flow),
        ['Year', 'Commodity Code', 'Trade Value Pct', 'Trade Value (US$)']
    ].rename(columns={
        'Year': 'year',
        'Commodity Code': 'ccode',
        'Trade Value Pct': 'pct',
        'Trade Value (US$)': 'value'
    }).to_dict(orient='records')
    
    return recs

In [54]:
commodities_import_export_data = {}

for country in np.unique(goods_data.Partner):
    if country != 'Other Asia, nes':
        commodities_import_export_data[country] = {}
        for flow in ['Import', 'Export']:
            commodities_import_export_data[country][flow] = goods_recs(country, flow)

In [55]:
with open('data/commodities_import_export_data.json', 'w') as f:
    json.dump(commodities_import_export_data, f)

#### SAVE AS SEPARATE FILES

In [61]:
# Partner - code map
partner_codes = data_pct.groupby('Partner').agg({
    'Partner Code': 'first'
}).to_dict()['Partner Code']

In [63]:
with open('data/commodities_flow/partner_codes.json', 'w') as f:
    json.dump(partner_codes, f)

In [65]:
for country in np.unique(goods_data.Partner):
    if country != 'Other Asia, nes':
        commodities_flow = {}
        country_code = partner_codes[country]
        
        for flow in ['Import', 'Export']:
            commodities_flow[flow] = goods_recs(country, flow)
        
        filename = 'data/commodities_flow/{country_code}.json'.format(country_code=country_code)
        with open(filename, 'w') as f:
            json.dump(commodities_flow, f)